In [2]:
# Import dependencies
import pandas as pd
import requests
import gmaps
import json
from config import g_key

In [3]:
file_to_load = pd.read_csv("weather_data/cities.csv")
city_data_df = pd.DataFrame(file_to_load)
city_data_df.head()

,City_ID,City,Country,Date,Latitude,Longitude,Max Temp,Humidity,Clouds,Wind
0,0,Alofi,NU,2022-09-16 03:48:09,-19.0595,-169.9187,80.49,69,75,11.50
1,1,Hermanus,ZA,2022-09-16 03:48:09,-34.4187,19.2345,55.80,91,98,3.65
2,2,Sakakah,SA,2022-09-16 03:48:10,29.9697,40.2064,78.37,25,0,5.66
3,3,Lompoc,US,2022-09-16 03:46:27,34.6391,-120.4579,63.01,88,100,13.80
4,4,Punta Arenas,CL,2022-09-16 03:47:04,-53.1500,-70.9167,32.11,69,0,5.75


In [4]:
# One caveat to using gmaps: 
# The data we use for any mapping must be either an integer or a floating-point decimal number.
# Check the dtypes of columns
city_data_df.dtypes

City_ID        int64
City          object
Country       object
Date          object
Latitude     float64
Longitude    float64
Max Temp     float64
Humidity       int64
Clouds         int64
Wind         float64
dtype: object

In [5]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [6]:
# 1. Assign the locations to an array of latitude and longitude pairs.
locations = city_data_df[["Latitude","Longitude"]]
print(len(locations))
# 2. Assign the weights variable to some values.
temperatures = city_data_df["Max Temp"] # an array of length equal to the locations array length
print(len(temperatures))
# 3. Assign the figure variable to the gmaps.figure() attribute.
fig = gmaps.figure()

# 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations.
heatmap_layer = gmaps.heatmap_layer(locations, weights=temperatures)

# 5. Add the heatmap layer.
fig.add_layer(heatmap_layer)

# 6. Call the figure to plot the data.
fig

531
531


Figure(layout=FigureLayout(height='420px'))

### Adjust Heatmap Zoom, Intensity, and Point Radius

In [7]:
# Add the geographic center of Earth in the form of latitude and longitude (30.0° N and 31.0° E). 
# Also, add a zoom level so that only one map of Earth is shown.
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)

# 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations.
heatmap_layer = gmaps.heatmap_layer(locations, weights=temperatures)

# 5. Add the heatmap layer.
fig.add_layer(heatmap_layer)

# 6. Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

### Create a Percent Humidity Heatmap

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Latitude", "Longitude"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
locations = city_data_df[["Latitude","Longitude"]]
clouds = city_data_df["Clouds"]
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
locations = city_data_df[["Latitude","Longitude"]]
wind_speed = city_data_df["Wind"]
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind_speed)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Get Vacation Criteria by asking customer

In [24]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip ?"))
max_temp = float(input("What is the maximum temperature you would like for your trip ?"))

What is the minimum temperature you would like for your trip ?35
What is the maximum temperature you would like for your trip ?75


In [25]:
city_data_df.head()

,City_ID,City,Country,Date,Latitude,Longitude,Max Temp,Humidity,Clouds,Wind
0,0,Alofi,NU,2022-09-16 03:48:09,-19.0595,-169.9187,80.49,69,75,11.50
1,1,Hermanus,ZA,2022-09-16 03:48:09,-34.4187,19.2345,55.80,91,98,3.65
2,2,Sakakah,SA,2022-09-16 03:48:10,29.9697,40.2064,78.37,25,0,5.66
3,3,Lompoc,US,2022-09-16 03:46:27,34.6391,-120.4579,63.01,88,100,13.80
4,4,Punta Arenas,CL,2022-09-16 03:47:04,-53.1500,-70.9167,32.11,69,0,5.75


In [26]:
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] > min_temp)]

# Drop the null values
new_preferred_cities_df = preferred_cities_df.dropna()
new_preferred_cities_df.count()

City_ID      361
City         361
Country      361
Date         361
Latitude     361
Longitude    361
Max Temp     361
Humidity     361
Clouds       361
Wind         361
dtype: int64

### Get Travel Destinations

In [27]:
# Create DataFrame hotel_df to store hotel names along with city, country, max temp, coordinates.
hotel_df = new_preferred_cities_df[["City","Country","Max Temp","Latitude","Longitude"]]

In [28]:
hotel_df["Hotel Name"] = ""

C:\Users\neera\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [29]:
hotel_df.head()

,City,Country,Max Temp,Latitude,Longitude,Hotel Name
1,Hermanus,ZA,55.80,-34.4187,19.2345,
3,Lompoc,US,63.01,34.6391,-120.4579,
5,Mahebourg,MU,73.26,-20.4081,57.7000,
6,Urusha,RU,54.28,54.0500,122.9000,
7,Guerrero Negro,MX,70.05,27.9769,-114.0611,


### Retrieve Hotels from a Nearby Search

In [30]:
# Using Google Places API

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Latitude"]
    lng = row["Longitude"]

    # Add the latitude and longitude to location key for the params dictionary.
    location = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    params = {"radius": 5000,
        "types": "lodging",
        "location" : location,
        "key": g_key}

    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()

    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        #print(result)
    except (IndexError):
        print("Hotel Not Found....skipping")

hotel_df.head()

C:\Users\neera\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....skipping
Hotel Not Found....s

,City,Country,Max Temp,Latitude,Longitude,Hotel Name
1,Hermanus,ZA,55.80,-34.4187,19.2345,Misty Waves Boutique Hotel
3,Lompoc,US,63.01,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
5,Mahebourg,MU,73.26,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
6,Urusha,RU,54.28,54.0500,122.9000,Dom Otdykha Lokomotivnykh Brigad
7,Guerrero Negro,MX,70.05,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"


In [34]:
hotel_df

,City,Country,Max Temp,Latitude,Longitude,Hotel Name
1,Hermanus,ZA,55.80,-34.4187,19.2345,Misty Waves Boutique Hotel
3,Lompoc,US,63.01,34.6391,-120.4579,"Holiday Inn Express Lompoc, an IHG Hotel"
5,Mahebourg,MU,73.26,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
6,Urusha,RU,54.28,54.0500,122.9000,Dom Otdykha Lokomotivnykh Brigad
7,Guerrero Negro,MX,70.05,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
...,...,...,...,...,...,...
523,Maarianhamina,AX,46.26,60.0973,19.9348,Park Alandia Hotel
524,Tuatapere,NZ,48.04,-46.1333,167.6833,Ron and Tony's Bed & Breakfast
525,Ajdabiya,LY,71.83,30.7554,20.2263,منزلة الحاج سالم خليفة العيساويه الزواوي
529,Bulgan,MN,37.45,48.8125,103.5347,Unitel Булган салбар


### Create a Maximum Temperature Heatmap from a Hotel DataFrame

In [41]:
locations = hotel_df[["Latitude","Longitude"]]
temperatures = hotel_df["Max Temp"]

gmaps.configure(api_key=g_key)

fig = gmaps.figure(center=(30.0,31.0), zoom_level=2)

# Add popup marker for the city
info_box_template = """
                    <dl>
                        <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
                        <dt>City</dt><dd>{City}</dd>
                        <dt>Country</dt><dd>{Country}</dd>
                        <dt>Max Temp</dt><dd>{Max Temp}</dd>
                    </dl>
"""

info_box_content = [info_box_template.format(**row) for index,row in hotel_df.iterrows()]

heat_layer = gmaps.heatmap_layer(locations, weights=temperatures, 
                                 dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=info_box_content)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))